In [3]:
import pandas as pd

In [2]:
#concentration = pd.read_csv('../../data/2019/19_concentrations.csv')
labels = pd.read_csv('../../data/2019/19_yellow_labels.csv')
pit = pd.read_csv('../../data/2019/19_time_since_pit.csv')
#weather = pd.read_csv('../../data/weather_data/2019_weather.csv', sep=';')
#speed = pd.read_csv('../../data/2020/20_avg_speed.csv')

In [3]:
max_concentration = pd.DataFrame(columns=['Time', 'Max Concentration'])
temp_max = 0
for index, row in concentration.iterrows():
    sector = row[1]
    if sector == 1:
        temp_max = 0
    if row[2] > temp_max:
        temp_max = row[2]
    if sector == 13:
        max_concentration.loc[index] = [row[0], temp_max]

In [4]:
start_time = weather['TIME_UTC_SECONDS'].iloc[0]

with open('../../data/2019/19_weather_mod.csv', 'w') as f:
    f.write('Time,Air Temp,Track Temp,Wind Speed\n')
    for index, row in weather.iterrows():
        if 65299 < ((row[0]-start_time)//20 * 20) < 65461:
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, 55.5,row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,55.5,row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,55.5,row[3], row[6]))
        else:
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, row[2],row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,row[2],row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,row[2],row[3], row[6]))
weather_mod = pd.read_csv('../../data/2019/19_weather_mod.csv')

In [7]:
pit = pit[pit['Last Pit'] < 140]
agg = max_concentration.merge(pit, how='inner', on='Time')
agg = agg.merge(weather_mod, how='inner', on='Time')
# agg = agg.merge(speed, how='inner', on='Time')
agg = agg.merge(labels, how='inner', on='Time')
agg

,Time,Max Concentration,Last Pit,Air Temp,Track Temp,Wind Speed,Yellow
0,20.0,26.265752,0.333333,56.40008,57.00002,2.6,0
1,40.0,18.707483,0.666667,56.40008,57.00002,2.6,0
2,60.0,21.505376,1.000000,56.40008,57.00002,0.0,0
3,80.0,22.408964,1.333333,56.40008,57.00002,0.0,0
4,100.0,16.846361,1.666667,56.40008,57.00002,0.0,0
...,...,...,...,...,...,...,...
3243,79200.0,8.333333,55.860050,56.89994,57.00002,3.2,0
3244,79220.0,2.688172,56.193383,56.89994,57.00002,3.2,0
3245,79240.0,4.201681,56.526717,56.89994,57.00002,4.6,0
3246,79260.0,2.801120,56.860050,56.89994,57.00002,4.6,0


In [6]:
agg.to_csv('../../data/2019/19_agg_data.csv', index=False)

In [1]:
times = [7560.0]
times = [t//60*60 for t in times]
MAXTIME = 9749
with open('../../data/2019/19_flag_times.csv', 'w') as f:
    f.write('Time,Flag\n')
    for i in range(0,MAXTIME,60):
        if i+120 in times:
            f.write('%s,1\n'%(i))
            f.write('%s,1\n'%(i+60))
            f.write('%s,1\n'%(i+120))
            i = i + 120
        else:
            f.write('%s,0\n'%(i))


In [4]:
labels = pd.read_csv('../../data/2019/19_flag_times.csv')
risk = pd.read_csv('../../data/2019/19_risk_score.csv')
sus = pd.read_csv('../../data/2019/19_sus_deriv.csv')
agg = risk.merge(sus, how='inner', on='Time')
agg = agg.merge(labels, how='inner', on='Time')
agg.to_csv('../../data/2019/19_agg_score.csv', index=False)